## Data Preparation

In [34]:
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
from pymongo import MongoClient
import json, re
from re import sub
from decimal import Decimal
import dask.dataframe as dd
from dask.distributed import Client, progress
import dask.dataframe as dd
from dask.delayed import delayed

### Dask Client

In [35]:
client = Client(n_workers=4, threads_per_worker=1)
client

/home/stiven/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:74: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:44907 Dashboard: http://127.0.0.1:40779/status,Cluster Workers: 4 Cores: 4 Memory: 8.26 GB


### Database Connection

In [36]:
DB_USER = 'anutibara'
DB_PASS = 'anutibara'
DB_HOST = 'scraping-cluster-7dtgt.gcp.mongodb.net'
DB_NAME = 'test_scraping_db'

try:
    client = MongoClient(f'mongodb+srv://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}')
    print("Database connected successfully")
except Exception as e:
    print("Error to connect to database: ", e)
db = client.get_database(DB_NAME)
properties = db.properties

Database connected successfully


### Database Queries

In [37]:
properties.count_documents({})

68

### Convert Mongo Collection to DataFrame

#### New Properties

In [38]:
new_properties_json = list(properties.find({ 'use': 'Nuevo' }))

In [39]:
df_general_info = json_normalize(new_properties_json, record_path='offersType', 
                                meta=['_id', 
                                    'urlProperty',
                                    'scrapingDate', 
                                    'scrapingHour', 
                                    'modifyDate',
                                    'modifyHour', 
                                    'code', 
                                    'status', 
                                    'type', 
                                    'use', 
                                    'nameProject', 
                                    'description'])
ddf_general_info = dd.from_pandas(df_general_info, npartitions=10)

In [40]:
df_location = json_normalize(new_properties_json, record_path='location', meta='urlProperty')
ddf_location = dd.from_pandas(df_location, npartitions=10)

In [41]:
ddf_1 = dd.merge(ddf_general_info, ddf_location, on='urlProperty')

In [42]:
df_owner_property = json_normalize(new_properties_json, record_path='ownerProperty', meta='urlProperty')
ddf_owner_property = dd.from_pandas(df_owner_property, npartitions=10)

In [43]:
ddf_2 = dd.merge(ddf_1, ddf_owner_property, on='urlProperty')

In [44]:
df_features = json_normalize(new_properties_json, record_path='features', meta='urlProperty')
ddf_features = dd.from_pandas(df_features, npartitions=10)

In [45]:
ddf_features = ddf_features.rename(columns={
    'price':'range_prices',
    'rooms': 'general_rooms',
    'bathrooms': 'general_bathrooms',
    'privateArea': 'range_private_area'
})

In [46]:
ddf_3 = dd.merge(ddf_2, ddf_features, on='urlProperty')

In [47]:
df_more_features = json_normalize(new_properties_json, record_path='moreFeatures', meta='urlProperty')
ddf_more_features = dd.from_pandas(df_more_features, npartitions=10)

In [48]:
ddf_4 = dd.merge(ddf_3, ddf_more_features, on='urlProperty')

In [49]:
ddf_new_properties = ddf_4.loc[:,~ddf_4.columns.duplicated(keep='first')]

In [50]:
ddf_new_properties = ddf_new_properties.dropna()

#### Old Properties

In [51]:
old_properties_json = list(properties.find({ 'use': 'Usado' }))

In [52]:
df_general_info = pd.DataFrame(old_properties_json, 
                                    columns=['_id', 
                                    'urlProperty', 
                                    'scrapingDate', 
                                    'scrapingHour', 
                                    'modifyDate',
                                    'modifyHour', 
                                    'code', 
                                    'status', 
                                    'type', 
                                    'use', 
                                    'nameProject', 
                                    'description'])
ddf_general_info = dd.from_pandas(df_general_info, npartitions=10)

In [53]:
df_location = json_normalize(old_properties_json, 'location')
ddf_location = dd.from_pandas(df_location, npartitions=10)

In [54]:
df_owner_property = json_normalize(old_properties_json, 'ownerProperty')
ddf_owner_property = dd.from_pandas(df_owner_property, npartitions=10)

In [55]:
df_features = json_normalize(old_properties_json, 'features')
ddf_features = dd.from_pandas(df_features, npartitions=10)

In [56]:
df_more_features = json_normalize(old_properties_json, 'moreFeatures')
ddf_more_features = dd.from_pandas(df_more_features, npartitions=10)

In [57]:
ddf_list = [ddf_general_info, ddf_location, ddf_owner_property, ddf_features, ddf_more_features]
ddf = dd.concat(ddf_list, axis=1)
ddf.reset_index()

,index,_id,urlProperty,scrapingDate,scrapingHour,modifyDate,modifyHour,code,status,type,use,nameProject,description,address,city,country,department,latitude,longitude,neighborhood,sector,contractType,financing,id,name,schedule,admonPrice,antiquity,bathrooms,condition,constructionArea,floor,garages,includesAdministration,interiorFloors,price,privateArea,rooms,squareMeters,squareMetersPrice,stratum,weather,exteriorFeatures,interiorFeatures,sectorFeatures
npartitions=19,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,object,object,object,object,int64,object,object,object,object,object,object,object,object,object,float64,float64,object,object,object,object,int64,object,object,float64,object,int64,object,object,object,object,object,int64,object,object,int64,object,float64,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [58]:
ddf_old_properties = ddf.loc[:,~ddf.columns.duplicated()]

In [59]:
ddf_old_properties = ddf_old_properties.dropna()

### Rename DataFrame Columns

#### New Properties

In [60]:
ddf_new_properties = ddf_new_properties.rename(columns={
                        "_id": "id_mongoose", 
                        "urlProperty":"id_property",
                        "scrapingDate": "scraping_date",
                        "scrapingHour": "scraping_hour",
                        "modifyDate": "modify_date",
                        "modifyHour": "modify_hour",
                        "status": "active",
                        "use": "new_property",
                        "nameProject": "name_project",
                        "offersType": "offers_type",
                        "id": "id_owner_property",
                        "name": "name_owner_property",
                        "contractType": "contract_type_owner_property",
                        "financing": "financing_owner_property",
                        "schedule": "schedule_owner_property",
                        "squareMeters": "range_square_meters",
                        "constructionArea": "range_construction_area",
                        "squareMetersPrice": "square_meters_price",
                        "interiorFloors": "interior_floors",
                        "includesAdministration": "includes_administration",
                        "admonPrice": "admon_price",
                        "interiorFeatures": "interior_features",
                        "exteriorFeatures": "exterior_features",
                        "sectorFeatures": "sector_features",
                        "offerType": "offer_type",
                        "privateArea": "private_area"
})

#### Old Properties

In [61]:
ddf_old_properties = ddf_old_properties.rename(columns={
                        "_id": "id_mongoose", 
                        "urlProperty":"id_property",
                        "scrapingDate": "scraping_date",
                        "scrapingHour": "scraping_hour",
                        "modifyDate": "modify_date",
                        "modifyHour": "modify_hour",
                        "status": "active",
                        "use": "new_property",
                        "nameProject": "name_project",
                        "offersType": "offers_type",
                        "id": "id_owner_property",
                        "name": "name_owner_property",
                        "contractType": "contract_type_owner_property",
                        "financing": "financing_owner_property",
                        "schedule": "schedule_owner_property",
                        "squareMeters": "square_meters",
                        "privateArea": "private_area",
                        "constructionArea": "construction_area",
                        "squareMetersPrice": "square_meters_price",
                        "interiorFloors": "interior_floors",
                        "includesAdministration": "includes_administration",
                        "admonPrice": "admon_price",
                        "interiorFeatures": "interior_features",
                        "exteriorFeatures": "exterior_features",
                        "sectorFeatures": "sector_features"
})

### Data Cleaning

#### New Properties

In [62]:
ddf_new_properties['id_mongoose'] = ddf_new_properties['id_mongoose'].astype(str)
ddf_new_properties['code'] = ddf_new_properties['code'].astype(int)
ddf_new_properties['active'] = ddf_new_properties['active'].apply(lambda status: True if (status == 'Active') else False, meta=('active', 'bool'))
ddf_new_properties['new_property'] = ddf_new_properties['new_property'].apply(lambda status: True if (status == 'Nuevo') else False, meta=('new_property', 'bool'))
ddf_new_properties['includes_administration'] = ddf_new_properties['includes_administration'].apply(lambda status: True if (status == 'Nuevo') else False, meta=('includes_administration', 'bool'))
ddf_new_properties['garages'] = ddf_new_properties['garages'].replace('', value = 0, regex = True).astype(int)
ddf_new_properties['stratum'] = ddf_new_properties['stratum'].replace('', value = 0, regex = True).astype(int)
ddf_new_properties['floor'] = ddf_new_properties['floor'].astype(int)
ddf_new_properties['area'] = ddf_new_properties['area'].astype(float)
ddf_new_properties['private_area'] = ddf_new_properties['private_area'].replace('', value = 0, regex = True)
ddf_new_properties['private_area'] = ddf_new_properties['private_area'].astype(float)
ddf_new_properties['rooms'] = ddf_new_properties['rooms'].astype(int)
ddf_new_properties['bathrooms'] = ddf_new_properties['bathrooms'].astype(int)
ddf_new_properties['price'] = ddf_new_properties['price'].apply(lambda price: Decimal(sub(r'[^\d,]', '', price)), meta=('price', 'int'))
ddf_new_properties['price'] = ddf_new_properties['price'].astype(int)

#### Old Properties

In [63]:
ddf_old_properties['id_mongoose'] = ddf_old_properties['id_mongoose'].astype(str)
ddf_old_properties['code'] = ddf_old_properties['code'].astype(int)
ddf_old_properties['active'] = ddf_old_properties['active'].apply(lambda status: True if (status == 'Active') else False, meta=('active', 'bool'))
ddf_old_properties['new_property'] = ddf_old_properties['new_property'].apply(lambda status: True if (status == 'Nuevo') else False, meta=('new_property', 'bool'))
ddf_old_properties['includes_administration'] = ddf_old_properties['includes_administration'].apply(lambda status: True if (status == 'Nuevo') else False, meta=('includes_administration', 'bool'))
ddf_old_properties['garages'] = ddf_old_properties['garages'].replace('', value = 0, regex = True)
ddf_old_properties['garages'] = ddf_old_properties['garages'].replace('Más de 10', value = 0, regex = True)
ddf_old_properties['garages'] = ddf_old_properties['garages'].astype(int)
ddf_old_properties['stratum'] = ddf_old_properties['stratum'].replace('', value = 0, regex = True)
ddf_old_properties['stratum'] = ddf_old_properties['stratum'].replace('Campestre', value = 0, regex = True)
ddf_old_properties['stratum'] = ddf_old_properties['stratum'].astype(int)
ddf_old_properties['price'] = ddf_old_properties['price'].apply(lambda price: Decimal(sub(r'[^\d,]', '', price)), meta=('price', 'int'))
ddf_old_properties['price'] = ddf_old_properties['price'].astype(int)
ddf_old_properties['square_meters'] = ddf_old_properties['square_meters'].str[0:-3]
ddf_old_properties['square_meters'] = ddf_old_properties['square_meters'].apply(lambda meters: Decimal(sub(r'[^\d,]', '', meters)), meta=('square_meters', 'float'))
ddf_old_properties['square_meters'] = ddf_old_properties['square_meters'].astype(float)
ddf_old_properties['private_area'] = ddf_old_properties['private_area'].str[0:-2]
ddf_old_properties['private_area'] = ddf_old_properties['private_area'].replace('', value = '0', regex = True)
ddf_old_properties['private_area'] = ddf_old_properties['private_area'].apply(lambda area: Decimal(sub(r'[^\d.]', '', area)), meta=('private_area', 'float'))
ddf_old_properties['private_area'] = ddf_old_properties['private_area'].astype(float)
ddf_old_properties['construction_area'] = ddf_old_properties['construction_area'].str[0:-3]
ddf_old_properties['construction_area'] = ddf_old_properties['construction_area'].apply(lambda area: Decimal(sub(r'[^\d,]', '', area)), meta=('construction_area', 'float'))
ddf_old_properties['construction_area'] = ddf_old_properties['construction_area'].astype(float)
ddf_old_properties['floor'] = ddf_old_properties['floor'].astype(int)

### Testing

#### New Properties

In [66]:
ddf_new_properties.describe().head()

AttributeError: 'function' object has no attribute 'head'

#### Old Properties

In [65]:
ddf_old_properties.describe().head()

,code,latitude,longitude,id_owner_property,admon_price,bathrooms,construction_area,floor,garages,interior_floors,price,private_area,rooms,square_meters,square_meters_price,stratum
count,4.800000e+01,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,4.800000e+01,48.000000,48.000000,48.000000,4.800000e+01,48.000000
mean,4.893133e+06,5.447376,-66.128241,611481.187500,188472.916667,2.437500,139.625000,3.250000,1.250000,0.041667,2.870429e+08,78.582708,2.312500,139.625000,1.938519e+06,4.583333
std,3.002181e+05,2.080911,25.258628,470527.601596,368877.132136,1.236093,125.713559,5.138507,1.682197,0.201941,5.036299e+08,95.038431,1.475419,125.713559,2.367323e+06,1.723842
min,3.884646e+06,0.000000,-75.630013,47928.000000,0.000000,0.000000,23.000000,0.000000,0.000000,0.000000,6.500000e+05,0.000000,0.000000,23.000000,1.093750e+04,1.000000
25%,4.989329e+06,6.201599,-75.572136,345580.000000,0.000000,2.000000,89.000000,0.000000,0.000000,0.000000,2.700000e+06,29.000000,2.000000,89.000000,2.952381e+04,4.000000
